In [1]:
from urllib.request import urlopen
from pathlib import Path
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = Path(".") / "data"
IMG_DIR = DATA_DIR / "images"

In [3]:
def get_soup_page(url: str):
    page = urlopen(url)
    print(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return soup

def rename_img_title(num: int, zero_len=5) -> str:
    number: str = str(num)
    foundation: str = ("0"* zero_len)
    assert len(foundation) >= len(number)
    return foundation[:-len(number)] + number

## Соберём адреса аукционов

In [4]:
base_url ="https://www.menziesartbrands.com"
auction_results_url = base_url + "/auction/results"


In [5]:
soup = get_soup_page(auction_results_url)

https://www.menziesartbrands.com/auction/results


In [6]:
auctions_urls = []

for button in soup.find_all("div", {"class": "ars_button"}):
    for ref in button.find_all("a"):
        if ref.text.lower() == "view result by lot":
            auctions_urls.append(f"{base_url}{ref['href']}")


In [7]:
len(auctions_urls)

30

## Теперь из каждого аукциона вытащим информацию о проданных картинах

In [8]:
count = 1
arr = []
cols = ['price', 'artist', 'title', 'yearCreation', 'image', 'image_url']

In [9]:
for auctions_url in auctions_urls[:-1]:
    
    auction = get_soup_page(auctions_url)
    
    # вычислим диапазон страниц в аукционе
    # есть один пустой аукцион
    if auction.find("li", {"class": "pager__item--last"}):
        last_page_message = auction.find("li", {"class": "pager__item--last"}).find("a")["href"]
        result = re.fullmatch(re.compile("\?page=(\d+)"),last_page_message)
        last_page = int(result.groups()[0])

    else:
        last_page = 1
        continue
    
    for num_page in tqdm(range(1, last_page + 1)):

        page_soup = get_soup_page(auctions_url + f'?page={num_page}')
        
        div_class = page_soup.find("div", {"class": "views-element-container"})
        
        # нашли элементы сетки с изображениями
        boxes = div_class.find_all("div", {"class": "box-item"})
        
        # переобор по лотам аукциона
        for box in boxes:
            row = []
            
            # проверка есть ли изображение
            if box.find("img") is None:
                continue
            
            # проверка есть ли цена
            dirty_price = box.find("div", {"class": "box__hammer-bp"})
            if dirty_price is None:
                continue
             
            # Если есть необходимая информация скрапим данные в соотвествие с порядком: ['price', 'artist', 'title', 'yearCreation', 'image', 'image_url']
                
            dirty_price = dirty_price.text
            dirty_price = re.search(re.compile("\$\d+,?\d+"), dirty_price).group()
            dirty_price = dirty_price.replace("$", "")

            clear_price_usd = int(''.join(dirty_price.split(",")))
            row.append(clear_price_usd)

            artist_name = box.find("div", {"class": "box__artist-name"}).find("div", {"class": "field--name-field-artistname"}).find("a").text
            row.append(artist_name)

            title = box.find("div", {"class": "box__title"}).find("a").contents[0].strip()
            row.append(title)

            year_creation = box.find("div", {"class": "box__title"}).find("a").find("span", {"class": "date"})
            # some pictures are without year of creation
            if year_creation is not None:
                year_creation = year_creation.text
                
            # todo: сейчас скачивается наиболее ранняя дата для простоты парсинга. 
                year_creation = int(re.search("(\d\d\d\d)", year_creation).group())

            row.append(year_creation)

            adres = box.find("div", {"class": "box__image"}).find("a")['href']
            img_title = rename_img_title(count)
            urllib.request.urlretrieve(f"https://www.menziesartbrands.com{adres}", IMG_DIR/ f"{img_title}.jpg")

            row.append(f"{count}.jpg")
            row.append(adres)
            
            count += 1

            arr.append(row)
            
        print(f"page_num = {num_page}, img_count = {count}")

        df = pd.DataFrame(arr, columns=cols)
    df.to_csv('data.csv')
    print(f"auction_url = {auctions_url}, img_count = {count}")

df = pd.DataFrame(arr, columns=cols)
df.to_csv('data.csv')

https://www.menziesartbrands.com/catalogue-details/20309


  0%|                                                                                                                                                                    | 0/16 [00:00<?, ?it/s]

https://www.menziesartbrands.com/catalogue-details/20309?page=1


  6%|█████████▊                                                                                                                                                  | 1/16 [00:13<03:20, 13.34s/it]

page_num = 1, img_count = 10
https://www.menziesartbrands.com/catalogue-details/20309?page=2


...

page_num = 20, img_count = 3009
https://www.menziesartbrands.com/catalogue-details/4483?page=21


 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉       | 21/22 [05:23<00:15, 15.26s/it]

page_num = 21, img_count = 3018
https://www.menziesartbrands.com/catalogue-details/4483?page=22


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [05:29<00:00, 14.99s/it]

page_num = 22, img_count = 3021
auction_url = https://www.menziesartbrands.com/catalogue-details/4483, img_count = 3021


In [15]:
df.to_csv(DATA_DIR/"menziesartbrands_data.csv", index=False)

In [13]:
df.tail()

,price,artist,title,yearCreation,image,image_url
3015,2375,Helen Ogilvie,Chooks in the Straw,1932.0,3016.jpg,/items/chooks-straw-0
3016,4375,John Olsen,Frog Surprised,2002.0,3017.jpg,/items/frog-surprised
3017,1375,Sidney Nolan,The Pursuit (Ned Kelly series),1971.0,3018.jpg,/items/pursuit-ned-kelly-series-0
3018,5250,Sidney Nolan,Quilting the Armour (Ned Kelly series),1970.0,3019.jpg,/items/quilting-armour-ned-kelly-series
3019,8750,BRETT WHITELEY,Warming and Reading,1981.0,3020.jpg,/items/warming-and-reading


## Ручная фильтрация 